# Word2Vec

En este notebook, vamos a crear un modelo Word2Vec utilizando texto de un artículo de la wikipedia.

Para ello, lo primero que vamos a hacer es instalar las dependencias necesarias. Necesitaremos `beautifulsoup` para hacer el *scrapping* del texto y `lxml` para parsear el código `html` que nos encontremos y poder extraer, por ejemplo, únicamente los párrafos.

In [ ]:
!pip install beautifulsoup4
!pip install lxml

Ahora realizaremos los imports necesarios:

In [ ]:
import bs4 as bs
import urllib.request
import re
import nltk

Y seguidamente descargaremos los paquetes que `nltk` necesita para funcionar:

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

Ya tenemos el software necesario preparado. Ahora vamos a escoger una página de la Wikipedia para obtener todos los párrafos que se encuentren en ella y utilizarlos como nuestro *corpus*.

Para ello, utilizaremos la librería `urllib` de Python:

In [ ]:
datos_wikipedia = urllib.request.urlopen('https://en.wikipedia.org/wiki/Natural_language_processing')

Para evitar problemas, necesitaremos convertir el texto en una codificación UTF-8:

In [ ]:
articulo = datos_wikipedia.read().decode('utf-8')

Y ahora extraeremos todos los párrafos que existan en la página:

In [ ]:
parser = bs.BeautifulSoup(articulo, 'lxml')
parrafos = parser.find_all('p')

Seguidamente, los concatenaremos en un único string:

In [ ]:
texto = ""
for p in parrafos:
    texto += p.text

In [ ]:
texto

## Pre-procesado

Como ya sabemos, es muy importante limpiar el texto y eliminar las stop-words.

### Limpieza del texto

Vamos a convertir el texto a minúsculas, luego a sustituir todos los caracteres que no sean letras por espacios, y por último a sustituir los espacios de forma que solo quede uno.

In [ ]:
texto = texto.lower()
texto

In [ ]:
texto = re.sub(r'[^a-z]', ' ', texto)
texto

In [ ]:
texto = re.sub(r'\s+', ' ', texto)
texto

In [ ]:
palabras = nltk.word_tokenize(texto)
palabras

In [ ]:
print(f'Número de palabras: {len(palabras)}')

Ahora que ya tenemos las palabras extraidas, podemos hacer un poco más de limpieza, eliminando las stop-words, como ya sabemos:

In [ ]:
from nltk.corpus import stopwords
palabras = [p for p in palabras if p not in stopwords.words('english')]
palabras

In [ ]:
print(f'Número de palabras: {len(palabras)}')

Fijaos cómo hemos conseguido limpiar bastante el dataset, quitando practicamente 500 palabras que no aportan información (las stop-words).

Sin embargo, si os fijáis, siguen habiendo cosas que no debería haber, como por ejemplo letras sueltas.

Vamos a inspeccionar un poco:

In [ ]:
for p in palabras:
    if len(p) < 3:
        print(p)

Como podemos observar, tenemos "palabras" que son solo las letras "e", "g" y "r", y además "ai". Lógicamente "AI" es una palabra y no deberíamos eliminarla (las siglas de Artificial Intelligence), pero las demás si podemos quitarlas:

In [ ]:
palabras = [p for p in palabras if p not in ['e', 'g', 'r']]
palabras

Parece que ya no están presentes, pero vamos a asegurarnos:

In [ ]:
for p in palabras:
    if len(p) < 3:
        print(p)

¡Perfecto! Pues ya podemos comenzar con el embedding Word2Vec. Para ello, podemos utilizar una librería llamada `gensim` que ya implementa este modelo y otros (más información en https://radimrehurek.com/gensim/).

La importamos y creamos el objeto `Word2Vec` con las palabras que acabamos de limpiar.

El parámetro `min_count` indica la frecuencia mínima que debe tener una palabra para que se incluya en el embedding. Esto quiere decir que si establecemos `min_count=2`, todas aquellas palabras que únicamente aparezcan una vez en nuestro texto, no se tendrán en cuenta en el embedding.

Por otra parte, el primer argumento (`sentences`) debe ser una lista de oraciones. Nosotros, como las hemos juntado anteriormente (por facilitar el pre-procesamiento), tenemos solo una, así que tendremos que usar `[palabras]` como argumento. Si no, dará error, podéis comprobarlo :)

¡Vamos al lio!

In [ ]:
from gensim.models import Word2Vec
word2vec = Word2Vec([palabras], min_count=2)

Ahora ya podemos comprobar qué tal funciona nuestro embedding. De acuerdo a lo que hemos visto en clase, deberia de ser capaz de encontrar palabras similares y distintas.

Pero antes, veamos realmente qué es lo que ha sucedido.

Por ejemplo, veamos cuál es la representación de la palabra "machine":

In [ ]:
word2vec.wv['machine']

In [ ]:
word2vec.wv['machine'].shape

Como podéis ver, es un vector de 100 dimensiones. Eso significa que hemos "embutido" (embebido, *embedded*) nuestras palabras en un espacio de 100 dimensiones.

Veamos ahora algunas palabras similares:

In [ ]:
palabras_similares = word2vec.wv.most_similar('machine')
for p in palabras_similares:
    print(p)

No parece muy coherente...

Hagamos un par de pruebas más para asegurarnos:

In [ ]:
palabras_similares = word2vec.wv.most_similar('artificial')
for p in palabras_similares:
    print(p)

In [ ]:
palabras_similares = word2vec.wv.most_similar('intelligence')
for p in palabras_similares:
    print(p)

Nada, definitivamente no parece funcionar muy bien.

¿Qué está pasando?

Pues que, por defecto, `Word2Vec` entrena el modelo subyacente (`CBOW` en nuestro caso, porque es el modelo por defecto y no hemos especificado lo contrario) durante 5 épocas. Esto puede ser poco, vamos a subirlo a 200.

También podemos modificar otros parámetros interesantes, como:

- `window`: el tamaño de la ventana de contexto. Por defecto es 5, vamos a subirlo a 7.
- `vector_size`: las dimensiones del embedding. Vamos a subirlas a 120.
- `sg`: 0 para modelo CBOW, 1 para modelo skip-gram.

Vamos a ejecutar el modelo CBOW con una ventana de 7 y un embedding de 120 dimensiones, y a entrenarlo durante 200 épocas, a ver si mejora lo anterior:

In [ ]:
from gensim.models import Word2Vec
word2vec = Word2Vec([palabras], min_count=2, window=7, vector_size=120, epochs=200)

Veamos ahora qué tal el embedding:

In [ ]:
palabras_similares = word2vec.wv.most_similar('machine')
for p in palabras_similares:
    print(p)

Parece un poco más coherente. Palabras como "learning", "learn", "deep", "modeling", "sequence", etc. tienen que ver con "machine".

Veamos las otras:

In [ ]:
palabras_similares = word2vec.wv.most_similar('artificial')
for p in palabras_similares:
    print(p)

In [ ]:
palabras_similares = word2vec.wv.most_similar('intelligence')
for p in palabras_similares:
    print(p)

Nada mal, ¿no os parece?

### Ejercicio

Construid un modelo Word2Vec, usando el modelo **skip-gram**, para la página misma página de Wikipedia que hemos usado en el ejemplo.

Comparad los resultados. ¿Qué opináis, funciona mejor o peor?

### Recursos:
- https://www.kaggle.com/code/vipulgandhi/bag-of-words-model-for-beginners
- https://towardsdatascience.com/how-to-train-a-word2vec-model-from-scratch-with-gensim-c457d587e031